In [1]:
import numpy as np
np.set_printoptions(linewidth=100)
import sympy as sm
import sympy.abc as sbl
from scipy.sparse import diags
import scipy.sparse.linalg as la
from sympy.utilities.lambdify import lambdastr
from collections import OrderedDict
import scipy.sparse as sp
sm.init_printing()
import time
from sympy import *
import matplotlib.pyplot as plt
import matplotlib

In [20]:
a = Function('a', integer=True)
i = Symbol('i')
N = Symbol('N', positive=True, integer=True)

sigma_ = (Sum(a(i)*2**i, (i, 0,20)) + 1)

In [28]:
(sigma_ * 3 + 1).doit() % 2  == 

a(0) mod 2

\begin{equation}
\\ \phi_0(x) = 
\begin{cases} 
\\ \frac{h - x}{h} & \:  h i  \leq x \leq h \left(i + 1\right) \\0 & \text{otherwise}
\end{cases}   \hspace{10mm} 
\end{equation}

\begin{equation}
\\ \phi_i(x) = 
\begin{cases} 
\frac{- h \left(i - 1\right) + x}{h} & \:  h \left(i - 1\right)  \leq x  \leq h i \\\frac{h \left(i + 1\right) - x}{h} &\:  h i  \leq x \leq h \left(i + 1\right) \\0 & \text{otherwise}
\end{cases}   \hspace{10mm} \text{for all i in 2 ... N- 1}
\end{equation}

\begin{equation}
\\ \phi_N(x) = 
\begin{cases} 
\frac{- h \left(N - 1\right) + x}{h} & \:  h \left(N - 1\right)  \leq x  \leq h N \\0 & \text{otherwise}
\end{cases}   \hspace{10mm} 
\end{equation}

In [3]:
h, x = sm.Symbol("h", positive=True), sm.Symbol("x", nonnegative=True)
N = 5

In [9]:
def phi(i, N = 6):
    
    if i == 0 :
        return sm.Piecewise( ( ( -x + (i+1)*h) / h, (x >=i*h) & (x <= (i+1)*h)) , \
                           (0, True) 

                      )
    elif i == N:
        return sm.Piecewise(  ((x - (i-1)*h) / h, (x >=(i-1)*h) & (x <= i*h))   , \
                   (0, True) 
                                       )             
    else:
        return sm.Piecewise(
                           ((x - (i-1)*h) / h, (x >=h*(i-1)) & (x <= i*h))    , \
                           ( ( -x + (i+1)*h) / h, (x >=i*h) & (x <= (i+1)*h)) , \
                           (0, True)  
                       )



In [10]:
#A int phi * phi
sm.Matrix(5, 5, lambda j, i: sm.integrate(phi(i, 4)*phi(j, 4), (x, 0, 4*h)))

⎡h   h              ⎤
⎢─   ─    0    0   0⎥
⎢3   6              ⎥
⎢                   ⎥
⎢h  2⋅h   h         ⎥
⎢─  ───   ─    0   0⎥
⎢6   3    6         ⎥
⎢                   ⎥
⎢    h   2⋅h   h    ⎥
⎢0   ─   ───   ─   0⎥
⎢    6    3    6    ⎥
⎢                   ⎥
⎢         h   2⋅h  h⎥
⎢0   0    ─   ───  ─⎥
⎢         6    3   6⎥
⎢                   ⎥
⎢              h   h⎥
⎢0   0    0    ─   ─⎥
⎣              6   3⎦

In [11]:

sm.Matrix(5, 5, lambda j, i: sm.integrate(phi(i, 4).diff(x)*phi(j, 4), (x, 0, 4*h)))

⎡-1/2  1/2    0     0     0 ⎤
⎢                           ⎥
⎢-1/2   0    1/2    0     0 ⎥
⎢                           ⎥
⎢ 0    -1/2   0    1/2    0 ⎥
⎢                           ⎥
⎢ 0     0    -1/2   0    1/2⎥
⎢                           ⎥
⎣ 0     0     0    -1/2  1/2⎦

In [12]:
#D int phi'x * phi'x
sm.Matrix(5, 5, lambda j, i: sm.integrate(phi(i, 4).diff(x)*phi(j, 4).diff(x), (x, 0, 4*h)))

⎡ 1   -1                ⎤
⎢ ─   ───   0    0    0 ⎥
⎢ h    h                ⎥
⎢                       ⎥
⎢-1    2   -1           ⎥
⎢───   ─   ───   0    0 ⎥
⎢ h    h    h           ⎥
⎢                       ⎥
⎢     -1    2   -1      ⎥
⎢ 0   ───   ─   ───   0 ⎥
⎢      h    h    h      ⎥
⎢                       ⎥
⎢          -1    2   -1 ⎥
⎢ 0    0   ───   ─   ───⎥
⎢           h    h    h ⎥
⎢                       ⎥
⎢               -1    1 ⎥
⎢ 0    0    0   ───   ─ ⎥
⎣                h    h ⎦

In [13]:
sm.Matrix(5, 5, lambda j, i: (phi(i, 4).diff(x)*phi(j, 4)).subs(x,4*h)) - sm.Matrix(5, 5, lambda j, i: (phi(i, 4).diff(x)*phi(j, 4)).subs(x,0))

⎡1  -1            ⎤
⎢─  ───  0   0   0⎥
⎢h   h            ⎥
⎢                 ⎥
⎢0   0   0   0   0⎥
⎢                 ⎥
⎢0   0   0   0   0⎥
⎢                 ⎥
⎢0   0   0   0   0⎥
⎢                 ⎥
⎢           -1   1⎥
⎢0   0   0  ───  ─⎥
⎣            h   h⎦

In [14]:
sm.Matrix(5, 5, lambda j, i: sm.integrate(phi(i, 4).diff(x)*phi(j, 4).diff(x), (x, 0, 4*h)))-(sm.Matrix(5, 5, lambda j, i: (phi(i, 4).diff(x)*phi(j, 4)).subs(x,4*h)) - sm.Matrix(5, 5, lambda j, i: (phi(i, 4).diff(x)*phi(j, 4)).subs(x,0)))

⎡ 0    0    0    0    0 ⎤
⎢                       ⎥
⎢-1    2   -1           ⎥
⎢───   ─   ───   0    0 ⎥
⎢ h    h    h           ⎥
⎢                       ⎥
⎢     -1    2   -1      ⎥
⎢ 0   ───   ─   ───   0 ⎥
⎢      h    h    h      ⎥
⎢                       ⎥
⎢          -1    2   -1 ⎥
⎢ 0    0   ───   ─   ───⎥
⎢           h    h    h ⎥
⎢                       ⎥
⎣ 0    0    0    0    0 ⎦

In [15]:
############end

In [16]:
def intgA(i, j):
    if i == j == 0:
         return sm.integrate(phi(i)**2, (x,(i)*h, (i+1)*h)) 
    elif i == j == N:
        return sm.integrate(phi(i)**2, (x,(i-1)*h, i*h))
    elif i == j:
        return sm.integrate(phi(i)**2, (x,(i-1)*h, i*h)) + sm.integrate(phi(i)**2, (x,(i)*h, (i+1)*h)) 
    elif i > j:
        return sm.integrate(phi(i) * phi(j), (x,(i-1)*h, i*h)) 
    else:
        return sm.integrate(phi(i) * phi(j),(x,(i)*h, (i+1)*h)) 
        

In [17]:
def intgB(k, i, j):
    if all([abs(i-j)  in [0, 1],  abs(j-k) in [0, 1], abs(k - i) in [0, 1]]):

        if i == j == k == 0:
             return sm.integrate(phi(k)*phi(i)*phi(j).diff(x), (x,(i)*h, (i+1)*h)) 
        elif i == j == k == N:
            return sm.integrate(phi(k)*phi(i)*phi(j).diff(x), (x,(i-1)*h, i*h))
        elif i == j == k:
            return sm.integrate(phi(k)*phi(i)*phi(j).diff(x), (x,(i-1)*h, i*h)) + sm.integrate(phi(k)*phi(i)*phi(j).diff(x), (x,(i)*h, (i+1)*h)) 
        elif i > j or i > k:
            return sm.integrate(phi(k)*phi(i)*phi(j).diff(x), (x,(i-1)*h, i*h)) 
        else:
            return sm.integrate(phi(k)*phi(i) * phi(j).diff(x),(x,(i)*h, (i+1)*h)) 
    else:
        return 0

In [18]:
a = sm.Matrix(N+1, N+1, lambda j, i: intgA(i, j))
ainv = a.inv()

In [19]:
[(phi(0)*phi(i).diff(x)).subs(x,0) for  i in range(6)]

In [ ]:
ainv

In [ ]:
sm.Matrix([phi(i) for i in range(6)]).T

In [7]:
def intgC(i, j):
    if i == j == 0:
         return sm.integrate(phi(i)*phi(j).diff(x), (x,(i)*h, (i+1)*h)) 
    elif i == j == N:
        return sm.integrate(phi(i)*phi(j).diff(x), (x,(i-1)*h, i*h))
    elif i == j:
        return sm.integrate(phi(i)*phi(j).diff(x), (x,(i-1)*h, i*h)) + sm.integrate(phi(i)*phi(j).diff(x), (x,(i)*h, (i+1)*h)) 
    elif i > j:
        return sm.integrate(phi(i)*phi(j).diff(x), (x,(i-1)*h, i*h)) 
    else:
        return sm.integrate(phi(i)*phi(j).diff(x),(x,(i)*h, (i+1)*h)) 
        

In [8]:
sm.Matrix(N+1, N+1, lambda i, j:intgC(i, j))

TypeError: phi() missing 1 required positional argument: 'N'

In [ ]:
def intgD(i, j):
    if i == j == 0:
         return sm.simplify(phi(i)*phi(j).diff(x))
    elif i == j == N:
        return sm.simplify(phi(i)*phi(j).diff(x))
    elif i == j:
        return sm.simplify(phi(i)*phi(j).diff(x)) + sm.simplify(phi(i)*phi(j).diff(x))
    elif i > j:
        return sm.simplify(phi(i)*phi(j).diff(x))
    else:
        return sm.simplify(phi(i)*phi(j).diff(x))
        

sm.Matrix(N+1, 1, lambda k, _:sm.Matrix(N+1, N+1, lambda i, j:intgB(k,i, j)))

sm.Matrix(N+1, N+1, lambda i, j:intgD(i, j))

sm.simplify(phi(0)*phi(0).diff(x))

In [ ]:
x, t, k, beta, i, j , h= sm.symbols("x t k beta i j H")
N = sm.symbols("N")

In [ ]:
u = t * sm.sin(sm. pi*x)
u

In [ ]:
expr = u.diff(t)  + beta* u.diff(x, 2) 

In [ ]:
sm.factor(expr)

In [ ]:

k = sm.abc.k

pos = sm.expand(sm.integrate(expr*((x-(k-1)*h)/h), (x, h*(k-1),h*k)))
neg = sm.expand(sm.integrate(expr*(-(x-(k+1)*h)/h), (x, h*(k),h*(k+1))))


In [ ]:
pos1= pos.coeff(t)
pos2= pos.coeff(t**2)



In [ ]:
pos1

In [ ]:
neg1= neg.coeff(t)
neg2= neg.coeff(t**2)

In [ ]:
neg1

In [ ]:

(lambdastr(k,       factor(simplify(neg1.subs(k, 0)))            ).replace("H","self.H").replace("beta","self.beta"), 
 
 lambdastr(k, factor(simplify(neg2.subs(k, 0)))     )   .replace("H","self.H").replace("beta","self.beta"))

In [ ]:
(lambdastr(k,sm.factor(simplify(pos1.subs(k, N).subs(N*h, 1)))).replace("H","self.H").replace("beta","self.beta"), 
 
 lambdastr(k,sm.factor(simplify(pos2.subs(k, N).subs(N*h, 1)))).replace("H","self.H").replace("beta","self.beta"))

In [ ]:

(lambdastr(k,       factor(simplify(neg1+pos1) )           ).replace("H","self.H").replace("beta","self.beta"), 
 
 lambdastr(k, factor(simplify(neg2+pos2)     ))   .replace("H","self.H").replace("beta","self.beta"))